# 3. 프롬프트 엔지니어링


In [ ]:
!pip install load_dotenv

In [ ]:
# dotenv로 설정
import os
from dotenv import load_dotenv

load_dotenv()

key = os.getenv("OPENAI_API_KEY")
print("Key length:", len(key) if key else "없음")

# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

Key length: 165


In [ ]:
from openai import OpenAI

client = OpenAI(api_key=key.strip())

try:
    resp = client.models.list()
    print("✅ 인증 성공:", resp.data[0].id)
except Exception as e:
    print("❌ 에러:", e)

In [ ]:
# import os
# from google.colab import userdata

# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

#### 【주의】알려진 오류에 대하여

openai 패키지가 의존하는 httpx의 업데이트로 인해, `openai==1.40.6`을 사용하는 부분에서 `TypeError: Client.__init__() got an unexpected keyword argument 'proxies'`라는 오류가 발생하게 되었습니다.

이 오류는 `!pip install httpx==0.27.2`와 같이 httpx의 특정 버전을 설치함으로써 회피할 수 있습니다.

또한, Google Colab에서 한 번 위 오류를 만난 후 `!pip install httpx==0.27.2`와 같이 패키지를 다시 설치한 경우, 다음 중 하나의 작업을 수행해야 합니다.

- Google Colab의 "런타임"에서 "세션 재시작"을 실행합니다
- "런타임 연결 해제 후 삭제"를 실행하고 패키지 설치부터 다시 시작합니다

In [19]:
!pip install openai==1.40.6 httpx==0.27.2

## 3.1 필요성

“프롬프트 엔지니어링”으로 100% 동일한 결과를 기대할수는 없지만 실용적인 비율로 지시에 따르게 할 수 있음

파인튜닝과 비교했을 때 비용 효율적

## 3.2. 프롬프트 엔지니어링

프롬프트 엔지니어링의 요소

- 프롬프트 설계: 모델에 적절한 지시를 제공하기 위한 문구나 구조를 고려
- 지시의 명확화: 모델에 요구하는 정보나 태스크 내용을 최대한 명확하게 전달하는 방법을 모색
- 피드백 루프: 출력 결과를 평가하고, 이를 바탕으로 프롬프트를 개선하는 과정을 반복

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "프롬프트 엔지니어링이란"},
    ],
)
print(response.choices[0].message.content)

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "질문에 100글자 정도로 답변해 주세요."},
        {"role": "user", "content": "프롬프트 엔지니어링이란"},
    ],
)
print(response.choices[0].message.content)

## 3.3. 프롬프트의 기본 구성 요소


### 프롬프트의 템플릿화


In [ ]:
prompt = '''\
다음 요리의 레시피를 생각해 주세요.

요리명: """
{dish}
"""
'''


def generate_recipe(dish: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt.format(dish=dish)},
        ],
    )
    return response.choices[0].message.content


recipe = generate_recipe("카레")
print(recipe)

In [ ]:
prompt = """\
사용자가 입력한 요리의 레시피를 생각해 주세요.

요리명: '''
{dish}
'''
"""


def generate_recipe(dish: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "사용자가 입력한 요리의 레시피를 생각해 주세요."},
            {"role": "user", "content": f"{dish}"},
        ],
    )
    return response.choices[0].message.content


recipe = generate_recipe("카레")
print(recipe)

### 출력 형식 지정하기


In [ ]:
system_prompt = """\
사용자가 입력한 요리의 레시피를 생각해 주세요.。

출력은 다음 JSON 형식으로 해주세요.

```
{
  "재료": ["재료1", "재료2"],
  "순서": ["순서1", "순서2"]
}
```
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "카레"},
    ],
)
print(response.choices[0].message.content)

## 3.4. 프롬프트 엔지니어링의 대표적인 기법


### Zero-shot 프롬프팅


In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "입력을 긍정적·부정적·중립 중 하나로 분류해 주세요.",
        },
        {
            "role": "user",
            "content": "ChatGPT는 매우 편리하다",
        },
    ],
)
print(response.choices[0].message.content)

### Few-shot 프롬프팅


In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "입력이 AI와 관련이 있는지 답변해 주세요."},
        {"role": "user", "content": "ChatGPT는 매우 편리하다"},
    ],
)
print(response.choices[0].message.content)

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "입력이 AI와 관련이 있는지 답변해 주세요."},
        {"role": "user", "content": "AI의 진화는 대단하다"},
        {"role": "assistant", "content": "true"},
        {"role": "user", "content": "오늘은 날씨가 좋다"},
        {"role": "assistant", "content": "false"},
        {"role": "user", "content": "ChatGPT는 매우 편리하다"},
    ],
)
print(response.choices[0].message.content)

### (칼럼) Few-shot 프롬프팅의 다른 형식


In [ ]:
prompt = """\
입력이 AI와 관련이 있는지 답변해 주세요.

Q: AI의 진화는 대단하다
A: true
Q: 오늘은 날씨가 좋다
A: false
Q: ChatGPT는 매우 편리하다
A:
"""

response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
)
print(response.choices[0].text)

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "입력이 AI와 관련이 있는지 답변해 주세요."},
        {"role": "system", "name": "example_user", "content": "AI의 진화는 대단하다"},
        {"role": "system", "name": "example_assistant", "content": "true"},
        {"role": "system", "name": "example_user", "content": "오늘은 날씨가 좋다"},
        {"role": "system", "name": "example_assistant", "content": "false"},
        {"role": "user", "content": "ChatGPT는 매우 편리하다"},
    ],
)
print(response.choices[0].message.content)

### Zero-shot Chain-of-Thought 프롬프팅


In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "답변만 한 마디로 출력해 주세요."},
        {"role": "user", "content": "10 + 2 * 3 - 4 * 2"},
    ],
)
print(response.choices[0].message.content)

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "단계별로 생각해 주세요."},
        {"role": "user", "content": "10 + 2 * 3 - 4 * 2"},
    ],
)
print(response.choices[0].message.content)